In [1]:
%load_ext autoreload
%autoreload 2
%load_ext snakeviz
%load_ext line_profiler

# turn off autoreload so that we can use the old model 
# when editing the current project

from coeditor.common import *
import os

os.chdir(proj_root())

In [2]:
import jedi, parso
from spot.static_analysis import PythonProject, UsageAnalysis
from coeditor.dataset import TokenizedEditDataset
from spot.utils import pretty_print_dict


In [6]:
from coeditor.code_change import *
from coeditor.history import get_commit_history

commits = get_commit_history(proj_root(), 10)
for c in commits[:3]:
    print(c)


CommitInfo(hash='d58fffa6f57746cb134bd98b5de3a8e668c4311d', parents=('d0fe36e93da8ca33161e524783335898bd47ebbc',), msg='Finish unchanged ref encoding.')
CommitInfo(hash='d0fe36e93da8ca33161e524783335898bd47ebbc', parents=('b2f78bf15287dca9f1312cdd7720e6e9175fdeef',), msg='- More analysis bug fix. - Encode unchanged using CSpan.')
CommitInfo(hash='b2f78bf15287dca9f1312cdd7720e6e9175fdeef', parents=('3fc7535603679d696adc71d9db9ffc11f0ba700e',), msg='Fix line ranges. Improve analysis and add tests.')


In [7]:
from coeditor.code_change import edits_from_commit_history, _tlogger
from coeditor.ctx_change_encoder import (
    CtxCodeChangeProblemGenerator,
    TkCtxCodeChangeEncoder,
    JediUsageAnalyzer,
)

generator = CtxCodeChangeProblemGenerator(JediUsageAnalyzer())
encoder = TkCtxCodeChangeEncoder()

workdir = proj_root() / "../temp-1"
# subprocess.run(["rm", "-r", workdir])
_tlogger.clear()
edits = edits_from_commit_history(
    proj_root(), commits, workdir, change_processor=generator, edit_encoder=encoder.encode_problem
)
print(f"{len(edits) = }")
print(f"{generator.analysis.error_counts=}")
display(_tlogger.as_dataframe())


processing commits: 100%|██████████| 9/9 [00:18<00:00,  2.04s/it]

len(edits) = 96
generator.analysis.error_counts={}


,name,count,avg_time,total_time
4,pre_edit_analysis,9,1.030676,9.276080
3,post_edit_analysis,9,0.561431,5.052877
6,change_encoder,9,0.181817,1.636356
1,parse_module,82,0.019240,1.577697
0,checkout,28,0.007753,0.217088
2,JModuleChange.from_modules,24,0.008065,0.193565
5,process_change,9,0.006110,0.054991


In [8]:
dset = TokenizedEditDataset.from_edits(edits)
pretty_print_dict(dset.overall_stats())
print(f"{generator.analysis.error_counts}")
for e in edits[9:10]:
    print(e.show())


n_projects: 1
n_edits: 96
n_additions: 0
input_tks:
   mean: 329.6
   median: 411
   min: 28
   max: 512
output_tks:
   mean: 63.646
   median: 43
   min: 3
   max: 256
n_references:
   mean: 7.9479
   median: 8
   min: 1
   max: 20
total_reference_tks:
   mean: 2643.6
   median: 2499.5
   min: 107
   max: 6964
{}
--------------------------------------------------------------------------------
path: coeditor.code_change/get_changed_spans
n_references: 9
total_reference_tks: 3312
========Ground Truth========
 <0>:<add>                     yield from recurse(sub_change, parent_changes)
 <4>:<del>                         code_change, scope_change, span.line_range, span.statements
 <5>:<add>                         code_change,
     <add>                         parent_changes,
     <add>                         span.line_range,
     <add>                         span.statements,
 <8>:<del>                     yield from recurse(s_change)
 <9>:<add>                     yield from recurse(s

In [6]:
%%timeit -n 1 -r 2
analysis = UsageAnalysis(pyproj, add_implicit_rel_imports=True, record_type_usages=True)

9.82 s ± 26 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [34]:
%%timeit -n 1 -r 2
PythonProject.from_root(proj_root())

4.79 s ± 21.6 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [17]:
jproj = jedi.Project(proj_root())
jmodules = dict()
for f in proj_root().glob("src/**/*.py"):
    s = jedi.Script(path=f, project=jproj)
    jmodules[f] = s._module_node


In [3]:
jproj = jedi.Project(proj_root())
s = jedi.Script(path=proj_root() / "src/coeditor/usage_analysis.py")
s.goto(28, 9)


[<Name name='self', description='param self'>]

In [5]:
from spot.static_analysis import PythonModule

%lprun -T output/lprof.txt -f PythonModule.from_cst pyproj = PythonProject.from_root(proj_root())


*** Profile printout saved to text file 'output/lprof.txt'. 


Timer unit: 1e-09 s

Total time: 20.8927 s
File: /home/jiayi/Projects/SPOT/src/spot/static_analysis.py
Function: from_cst at line 495

Line #      Hits         Time  Per Hit   % Time  Line Contents
   495                                               @staticmethod
   496                                               def from_cst(module: cst.Module, name: str, drop_comments: bool) -> "PythonModule":
   497        60 4516638349.0 75277305.8     21.6          wrapper = MetadataWrapper(module)
   498        60 8465727458.0 141095457.6     40.5          src_map = dict(wrapper.resolve(PositionProvider))
   499        60     151722.0   2528.7      0.0          module = wrapper.module
   500        60      18831.0    313.9      0.0          removed_comments = []
   501        60      19426.0    323.8      0.0          if drop_comments:
   502        60    6497932.0 108298.9      0.0              remover = CommentRemover(src_map=src_map)
   503        60 5250321636.0 87505360.6     25.1        